# Fingerprints to SMILES amb MolForge
Transforma una taula amb **fingerprints** en **SMILES** fent servir MolForge.

**Entrada**: CSV amb columna `fingerprints_input_ECPF4`.

**Sortida**: CSV amb columnes `fingerprints_input_ECFP4` i `SMILES_output_ECPF4`.

## Imports

In [1]:
# Per definir l'arrel del projecte
import os

# Pandas pels dataframes
import pandas as pd
import numpy as np

# Per cridar MolForge i guardar-ne l'output
from MolForge import main as molforge_main
import sys
import io
from contextlib import redirect_stdout

## Inputs (part a editar)

Arrel del projecte

In [2]:
os.chdir("/export/home/ddiestre/MolForge_Testing")

Paràmetres de MolForge

In [3]:
FP_NAME = "ECFP4"
MODEL_TYPE = "smiles"  # ["smiles", "selfies"]
DECODE = "greedy"  # ["greedy", "beam"]
CHECKPOINT_NAME = "ECFP4_smiles_checkpoint.pth"

Fitxer de fingerprints preprocessat (path a partir de MolForge_Testing/)

In [4]:
input_path = "data/MolForge_input/CoCoGraph_MFinput_1.csv"
in_col_name = "fingerprints_input_" + FP_NAME

Fitxer en que guardar l'output (path a partir de MolForge_Testing/)

In [5]:
output_path = "data/MolForge_output/CoCoGraph_MFoutput_1.csv"
out_col_name = "SMILES_output_" + FP_NAME

## 1. Lectura del fitxer

In [6]:
# Lectura del fitxer
df = pd.read_csv(input_path, sep = ',', index_col = 0)
df.head(6)

,SMILES_input,fingerprints_input_ECFP4
id,,
1,CCCCCN(CCCCc1c2cc3sc2nn13)C(=O)c1ccc(OCC)c(Cl)c1,70 80 85 94 162 237 294 347 366 378 412 425 51...
2,O=C(Nc1n[nH]c2ccc(Br)cc12)N1CCCC1,74 91 119 122 218 328 369 378 650 708 728 766 ...
3,CCN(C(=O)N(C)c1ccccc1OC)C(=Cc1ccc(OC)cc1)c1ccc...,25 54 80 249 252 294 322 376 431 650 694 695 7...
4,CC1(Cl)OC(=O)C1(C)Cl,99 113 314 650 656 667 1041 1057 1060 1135 127...
5,CCc1cc(C=CC(C)=Cc2cc(OC)ccc2C)ccc1C,25 31 80 135 294 322 517 650 694 695 718 781 8...
6,CCc1ccc2ccc3[nH]c(=O)n(C)c(=O)[nH]c(=O)c4ccc(c...,2 80 119 145 203 294 310 314 345 437 552 564 6...


## 2. Execució de Molforge

In [7]:
def run_for_fp(input_fp, fp_name, model_type, checkpoint_name, decode):
    # 0. Tractem els casos "NaN" i "InvalidSMILE"
    if (input_fp == "InvalidSMILE") or (input_fp is np.nan):
        return np.nan

    # 1. Guardar sys.argv original (per restaurar-lo després)
    original_argv = sys.argv
    
    # 2. Crear els arguments com si s'haguessin passat des de CLI (Command Line Interface)
    sys.argv = [
        "",  # el primer argument de sys.argv s'ignora
        f"--input={input_fp}",
        f"--fp={fp_name}",
        f"--model_type={model_type}",
        f"--checkpoint={checkpoint_name}",
        f"--decode={decode}",
    ]
    
    # 3. Capturar stdout
    buf = io.StringIO()  # creem un buffer text en memòria (on guardarem l'output de MolForge)
    try:  # per asegurar-nos que restaurem sys.argv encara que peti
        with redirect_stdout(buf):  # guardem l'output de molforge a buf
            molforge_main()  # executem molforge_main amb els arguments de sys.argv
    finally:
        # 4. Restaurar sys.argv (important en notebooks! tot i que no és estrictament necessari en aquest cas, és una bona pràctica)
        sys.argv = original_argv
    
    # 5. Processar resultat
    output = buf.getvalue()  # transforma buf en string
    for line in output.splitlines():
        line = line.strip()  # elimina espais i salts de línia al principi i al final
        if line.startswith("Result:"):
            return line.split("Result:", 1)[1].strip().replace(" ", "")  # separem per "Result:" i agafem només la part de després fent-li un strip i treient-li espais
    
    return None

In [8]:
smiles_out = [] # aquesta serà la nostra columna "out_col_name"
len_df = len(df)
for i in range(len_df): # de 0 a len_df-1
    s = run_for_fp(df[in_col_name][i+1], FP_NAME, MODEL_TYPE, CHECKPOINT_NAME, DECODE)
    smiles_out.append(s)
    print(f"\r[{i+1}/{len_df}]", end="", flush=True) # Seguiment del progrés

[10/10]

## 3. Guardar l'output

In [9]:
# Creem el nou dataframe
df[out_col_name] = smiles_out
df.head(6)

,SMILES_input,fingerprints_input_ECFP4,SMILES_output_ECFP4
id,,,
1,CCCCCN(CCCCc1c2cc3sc2nn13)C(=O)c1ccc(OCC)c(Cl)c1,70 80 85 94 162 237 294 347 366 378 412 425 51...,CCCCCC1=C2C=C3C=C(C(=NN3C(=C2C=C4N1N=C5N4N=C6C...
2,O=C(Nc1n[nH]c2ccc(Br)cc12)N1CCCC1,74 91 119 122 218 328 369 378 650 708 728 766 ...,C1CCN(C1)C(=O)NC2=NNC3=C2C=C(C=C3)Br
3,CCN(C(=O)N(C)c1ccccc1OC)C(=Cc1ccc(OC)cc1)c1ccc...,25 54 80 249 252 294 322 376 431 650 694 695 7...,CCN(C1=CC=CC=C1OC)C(=O)N(C)C2=CC=CC=C2/C(=C/C3...
4,CC1(Cl)OC(=O)C1(C)Cl,99 113 314 650 656 667 1041 1057 1060 1135 127...,CC1(C2(C(C(=O)O1)(C(C(=O)O2)(Cl)Cl)Cl)C)C
5,CCc1cc(C=CC(C)=Cc2cc(OC)ccc2C)ccc1C,25 31 80 135 294 322 517 650 694 695 718 781 8...,CCC1=C(C=CC(=C1)/C=C/C(=C/C2=C(C=CC(=C2)OC)C)/C)C
6,CCc1ccc2ccc3[nH]c(=O)n(C)c(=O)[nH]c(=O)c4ccc(c...,2 80 119 145 203 294 310 314 345 437 552 564 6...,CCC1=CC2=C(C=C1)NC3=C2C=CC4=C3C5=C(C=C4)C6=C(C...


In [10]:
df.to_csv(output_path)